In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from hep_VQAE import data_preprocessing as dp
import tensorflow as tf
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
import time

import warnings
warnings.filterwarnings('ignore')

2022-08-12 17:06:39.514370: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-12 17:06:39.514391: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Circuit

In [2]:
DATA_QBITS = 4
LATENT_QBITS = 2
TRASH_QBITS = DATA_QBITS - LATENT_QBITS
TOTAL_QBITS = DATA_QBITS + TRASH_QBITS + 1

In [3]:
dev1 = qml.device('qiskit.ibmq', wires=TOTAL_QBITS, backend='ibmq_qasm_simulator', ibmqx_token="b3d3c72c5444443a55aad7cb536d64eb91922e050d1d9f4180c86c73326a601ab32b433e1c57ff26c94e011018435962f5ed3b49c3f6918fb6cb0adfd1a89f72")

In [4]:
def layer(params, data, wires):
    for i in range(len(wires)):
        qml.RY(params[i*2] + params[i*2+1]*data[i] , wires=i)
    qml.CNOT(wires=[wires[-1],0])
    for i in range(len(wires)-1):
        qml.CNOT(wires=[i, i+1])


@qml.batch_input(argnum=0)
@qml.qnode(dev1)
def circuit(data, enc_params):
    #qml.AngleEmbedding(data, wires=range(DATA_QBITS))
    print("d")
    print(data)
    print("pr")
    print(enc_params)
    for i in range(LAYERS):
        for j in range(DATA_QBITS):
            qml.RY(enc_params[i*2*DATA_QBITS:(i+1)*2*DATA_QBITS][j*2] + enc_params[i*2*DATA_QBITS:(i+1)*2*DATA_QBITS][j*2+1]*data[j] , wires=j)
        qml.CNOT(wires=[DATA_QBITS-1,0])
        for i in range(DATA_QBITS-1):
            qml.CNOT(wires=[i, i+1])

        #layer(enc_params[i*2*DATA_QBITS:(i+1)*2*DATA_QBITS], data, [x for x in range(DATA_QBITS)])
    qml.Hadamard(wires=TOTAL_QBITS-1)
    for i in range(TRASH_QBITS):
        #c += cirq.ControlledGate(sub_gate=cirq.SWAP, num_controls=1).on(swap_qbit, reference_qbits[i], network_qbits[num_data_qbits - num_latent_qbits:][i])
        qml.CSWAP(wires=[TOTAL_QBITS - 1, LATENT_QBITS + i, DATA_QBITS + i])
    qml.Hadamard(wires=TOTAL_QBITS-1)
    return qml.expval(qml.PauliZ(TOTAL_QBITS-1))

In [5]:
#fig, ax = qml.draw_mpl(circuit)([1,11,1,1,1,1,1,1,1,1,1,1],[0.1,0.24,0.3,0.14])
#fig.show()

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

In [7]:
def filter_36(x, y):
    keep = (y == 3) | (y == 6)
    x, y = x[keep], y[keep]
    y = y == 3
    return x,y

x_train, y_train = filter_36(x_train, y_train)
x_test, y_test = filter_36(x_test, y_test)

In [8]:
DOWNSCALE = 2
DATA_DIMENSION = DOWNSCALE*DOWNSCALE
LATENT_BITS = 2

x_train, x_test = dp.PCA_reduce(x_train, DATA_DIMENSION, val_data=x_test)

In [9]:
x_train_threes = np.array(x_train[y_train==True])
x_train = np.array(x_train[y_train==False])

x_test_threes = np.array(x_test[y_test==True])
x_test = np.array(x_test[y_test==False])

In [10]:
LAYERS = 1

def cost(params, X):
    loss = 0.0
    for i in range(len(X)):
        f = circuit(params, X[i])
        loss = loss + (1 - f) ** 2
    return loss / len(X)

def cost_batch(params, X):
    loss = 0.0
    batch_res = circuit(params, X)
    print(type(batch_res))
    print("--")
    print(batch_res)
    loss = np.mean((1-batch_res)**2)
    print(loss)
    return loss

In [11]:
def iterate_minibatches(data, batch_size):
    for start_idx in range(0, data.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield data[idxs]

In [12]:
learning_rate = 0.01
epochs = 1
batch_size = 3

In [13]:
opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)
# opt = qml.QNGOptimizer(learning_rate)
#opt = GradientDescentOptimizer(learning_rate)

In [14]:
weight_shapes = {"weights": (LAYERS*2*DATA_QBITS)}
params = np.random.uniform(size=weight_shapes["weights"], requires_grad=True)

In [15]:
params.shape

(8,)

In [16]:
x_train = x_train[:50]
x_train_threes = x_train_threes[:50]

In [17]:
start = time.time()

for it in range(epochs):
    for j,Xbatch in enumerate(iterate_minibatches(x_train, batch_size=batch_size)):
        Xbatch = np.array(Xbatch, requires_grad=False)
        print("init_params")
        print(params)
        params = opt.step(cost_batch, params, Xbatch)
        print(j, end="\r")
        break
    break
    loss = cost(params, x_train)
    val_loss = cost(params, x_test[:100])
    
    print(f"Epoch: {it} | Loss: {loss} | Val Loss: {val_loss}")
    
end = time.time()

print(f"Time for {epochs} epochs with {LAYERS} layers: {end - start}")

init_params
[0.27785436 0.02686934 0.1945819  0.8669702  0.89783929 0.48320459
 0.12408604 0.10524092]
d
Autograd ArrayBox with value [0.27785436 0.02686934 0.1945819  0.8669702  0.89783929 0.48320459
 0.12408604 0.10524092]
pr
[[0.79680565 0.45641394 0.59183778 0.44915419]
 [0.52812885 0.37485367 0.31909504 0.52948142]
 [0.69171564 0.45219032 0.34722902 0.491524  ]]


IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
x_test_threes = x_test_threes[:100]
x_test = x_test[:100]

In [ ]:
pred_six = np.array([circuit(params,i) for i in x_test])
np.mean(pred_six)
print("Median six reconstruciton fidelities: ",np.median(pred_six))

pred_three = np.array([circuit(params,i) for i in x_test_threes])
np.mean(pred_three)
print("Median three reconstruciton fidelities: ",np.median(pred_three))

In [ ]:
bins = np.histogram(np.hstack((pred_six.reshape((pred_six.shape[0],)), pred_three.reshape((pred_three.shape[0],)))), bins=10)[1]
plt.hist(pred_six, histtype='step', label="sixes",bins=bins)
plt.hist(pred_three, histtype='step', label="threes",bins=bins)

In [ ]:
def acc_for_threshold(T):
    # sixes that are predicted six
    num_six_right = np.sum(pred_six > T)
    # threes that are predicted three
    num_three_right = np.sum(pred_three < T)
    acc = (num_six_right + num_three_right)/(len(pred_six) + len(pred_three))
    return acc

Ts = np.linspace(0,1,2000)
accs = []
for i in Ts:
    accs.append(acc_for_threshold(i))
print(max(accs))
plt.ylabel("Tagging accuracy")
plt.xlabel("Anomaly threshold")
plt.plot(Ts, accs)  